# Návod

Je potřeba mít vytvořený vlastní model YOLOv8, pomocí které provádíem objektovou detekci

*   jsou detekovány rostliny, které jsou na obrázku označeny červeným boxem a uloženy v dočasné složce. Adresu této složky je nutné zapsat ve tvaru (/content/JmenoProjektu/ + JmenoSložky), společně s označenými boxy se uloží do samostatné složky labels textové soubory, co obsahují souřadnice k červeným bounding boxům

*   Následně je do tabulky pandas jsou zapsány informace o obrázcích, tedy jméno obrázku, šířka, výška a informace o bounding boxech

*   Na základě těchto informací je vybrán u každé rostliny vhodný vzor, podle kterého se ořežou všechny obrázky této rostliny a výstřižky se uloží na vhodné místo. Složka s obrázky a labels se po ořezání vymaže.


**Pravidla, podle kterého se vybere vhodná rostlina:**
* šířka a výška nesmí být větší než 55
* nesmí předcházet obrázky co mají šířku a výšku menší než 55 (tedy berou se pouze první nálezy, jakmile se narazí na rostlinku co ma v výšku nebo šířku větší než 55, zbytek rostlin v souboru se bere v potaz)  
* z těchto rostlin, co splnují dvě předcházejcí pravidla, vyberem tu, co má největší confidence



In [ ]:
#if run locally or colab
import os
drive_path = ""
kit_ml_dir = "/KIT ML/Fenotypizace - vzchazeni"
if os.path.isdir("/Volumes/GoogleDrive/Shared drives"):
    drive_path = "/Volumes/GoogleDrive/Shared drives"
elif os.path.isdir("G:/Shared drives"):
    drive_path = "G:/Shared drives"
elif not os.path.isdir(drive_path):
    from google.colab import drive
    drive.mount('/content/gdrive')
    drive_path = "/content/gdrive/Shareddrives"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
gdrive_path = drive_path +"/" # TOHLE MOŽNÁ ZMĚNIT netuším, jak se připojí pc ke googlu

version_path="KIT ML 2/Data/cropped75-smazatt/"

base_dire = gdrive_path + version_path # tady nejduležitější proměnná

name= "detect" # tohle jméno radši neměnit neměnit

save_path=base_dire+"runs/"

# cesta pro model
path_to_model = gdrive_path + "KIT ML 2/Code/Final/2. Object detection/models/better_plants_1.1.pt"

orig_data_path = gdrive_path + "/KIT ML 2/Data/1-TraysSegmentation/segmented/"

label_path = save_path+"detect/labels"

path_of_csv = gdrive_path+ version_path +"/bounding_boxes/"

base_save_adress = gdrive_path + "/" + version_path

conf=0.6

print(f"gdrive_path:      {gdrive_path}")
print("")
print(f"base_dire:        {base_dire}")
print("")
print(f"name:             {name}")
print("")
print(f"save_path:        {save_path}")
print("")
print(f"path_to_model:    {path_to_model}")
print("")
print(f"orig_data_path:   {orig_data_path}")
print("")
print(f"label_path:       {label_path}")
print("")
print(f"conf:             {conf}")
print("")
print(f"base_save_adress:  {base_save_adress}")
print("")

gdrive_path:      /content/gdrive/Shareddrives/

base_dire:        /content/gdrive/Shareddrives/KIT ML 2/Data/cropped75-smazatt/

name:             detect

save_path:        /content/gdrive/Shareddrives/KIT ML 2/Data/cropped75-smazatt/runs/

path_to_model:    /content/gdrive/Shareddrives/KIT ML 2/Code/Final/2. Object detection/models/better_plants_1.1.pt

orig_data_path:   /content/gdrive/Shareddrives//KIT ML 2/Data/1-TraysSegmentation/segmented/

label_path:       /content/gdrive/Shareddrives/KIT ML 2/Data/cropped75-smazatt/runs/detect/labels

conf:             0.6

base_save_adress:  /content/gdrive/Shareddrives//KIT ML 2/Data/cropped75-smazatt/



In [ ]:
HOME = os.getcwd()
print(HOME)
!pip install ultralytics==8.0.20

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()
from ultralytics import YOLO

from IPython.display import display, Image
from PIL import Image
import shutil
import glob
import io

import numpy as np
import pandas as pd
import csv

import os
import shutil
import cv2


Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.3.0+cu121 CPU
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 30.1/107.7 GB disk)


In [ ]:
def predictImgsSINGLEDirectory(path_to_model: str, path_to_imgs: str, conf: int, save_path: str, name: str):
    """
    Predict images in a single directory using a YOLO model.

    Args:
        path_to_model (str): Path to the trained YOLO model.
        path_to_imgs (str): Path to the directory containing images for prediction.
        conf (int): Confidence threshold for predictions.
        save_path (str): Path to save the prediction results.
        name (str): Name for the project/folder where results will be saved.

    Returns:
        None
    """
    data_path = path_to_imgs
    model_path = path_to_model
    confi = conf
    project = save_path

# prediction using YOLOV8

    results = model.predict(
        source=path_to_imgs,
        save=True,
        project=project,
        name=name,
        conf=confi,
        save_txt=True,
        max_det=5,
        save_conf=True,
        line_thickness=1,
        hide_labels=True,
        hide_conf=True
    )

    print(project)
    print(name)

def delete(save_path: str):
    """
    odstraní složku a celý její obsah

    """
    useless_folder = save_path

    # Odstranění složky a jejího obsahu
    try:
        shutil.rmtree(useless_folder, ignore_errors=True)
    except FileNotFoundError:
        pass



def img_size(file_name: str, orig_data_path: str) -> tuple:
    """
    zjistí rozměry obrázku

    file_name = název obrázku, kde zjištujem rozměry

    je potřeba zde doplnit adresu k složkám s obrázky (orig_data_path), adresda k obrázkům se dál využívá k odvození adres obrázků

    return: width, height

    """
    img_name = file_name
    # rozdělíme img podle znaku "_", abychom mohli doplnit označení obrázku a mohla nám tak vznikonut cesta k obrázkům
    shards = img_name.split("_")
    shards[2] = shards[2].replace(".txt", "")
    img_path = orig_data_path + "Tray_" + shards[0] + "/" + shards[2] + "/" + img_name
    # protože nás zajímají jen obrázky, kde je detekce, používáme obrázky který mají textový soubor, kterým nahradíme příponu
    img_path = img_path.replace(".txt", ".png")

    img = Image.open(img_path)
    #print(img_path)
    width, height = img.size
    return width, height



def create_dataframe(label_path: str, delimiter: str = ' ') -> pd.DataFrame:
    """
    na základě textových souborů ve složce labels,což jsou souřadnice detekovaných bounding boxů se vytvoří tabulka
    se vytvoří tabulka ve formě dataframe, kde se všechny bounding boxy zapíší + názvy obrázků

    """
    # Získání seznamu názvů souborů v adresáři
    file_names = [f for f in os.scandir(label_path) if f.is_file()]

    # Inicializace prázdného seznamu pro data
    data = []

    # Procházení souborů v adresáři
    for file in file_names:
        file_path = file.path

        # Otevření souboru a čtení jeho obsahu jako CSV s určeným delimiterem
        df = pd.read_csv(file_path, delimiter=delimiter, header=None)

        # Pokud očekáváme, že každý soubor má alespoň 6 částí (kromě názvu souboru)
        if len(df.columns) >= 6:
            # Přidání názvu souboru jako první sloupec a následné sloupce kromě první položky
            for _, row in df.iterrows():
                data.append([file.name] + row.tolist()[1:])  # Přeskočení první položky

    # Vytvoření pandas DataFrame s odpovídajícími názvy sloupců
    header = ['name', 'center_x', 'center_y', 'width', 'height', 'conf']
    df = pd.DataFrame(data, columns=header)

    return df

def add_size_to_dataframe(df: pd.DataFrame, orig_data_path: str) -> pd.DataFrame:
    """
    přidá do dataframu původní výšku a šířku obrázku a přepočítá rozměry bounding boxu na půvondím obrázku
    to se udělá u všech obrázkl, co jsou zapsaný v df

    """
    # Procházení řádků DataFrame a výpočet nových hodnot
    for row in df.itertuples(index=True):
        index = row.Index
        file_name = row.name
        # Zde načteme šířku a výšku z původního obrázku
        width, height = img_size(file_name, orig_data_path)

        real_center_x = float(row.center_x) * float(width)
        real_center_y = float(row.center_y) * float(height)
        bb_width = float(row.width) * float(width)
        bb_height = float(row.height) * float(height)

        df.at[index, 'real_width'] = width
        df.at[index, 'real_height'] = height
        df.at[index, 'real_center_x'] = real_center_x
        df.at[index, 'real_center_y'] = real_center_y
        df.at[index, 'bb_width'] = bb_width
        df.at[index, 'bb_height'] = bb_height

    return df

def bb_adding_smallbox(df: pd.DataFrame) -> pd.DataFrame:
    """
    určí smallbox, to znamená že šířka výška jsou menší než 55

    """

    # Vytvoření sloupce 'smallBox' na základě podmínky
    df['smallBox'] = df.apply(lambda row: 'yes' if row['bb_width'] <= 55 and row['bb_height'] <= 55 else 'no', axis=1)
    return df

def bb_smallBox_edit(df: pd.DataFrame) -> pd.DataFrame:
    """
    Tato funkce zajistí, že pokud v detekci začneme narážet na obrázky větší než 55 v šířce nebo výšce,
    tak se označí jako nevhodné (smallBox = no) pro pozdější výběr vhodného vzoru.
    Zabrání nám to, aby se vhodný vzor byla velká rostlina, pak by se celý soubor vystřihl špatně.
    """
    boxAlready = False

    # Iterace přes řádky DataFrame
    for row in df.itertuples(index=True):
        index = row.Index
        if boxAlready:
            df.at[index, 'smallBox'] = "no"  # Přepsání hodnoty na "no" ve sloupci "smallBox"

        if row.smallBox == "no":
            boxAlready = True

    return df



def finding_BB_DF(df: pd.DataFrame) -> tuple:
    """
    tato funkce přijmá dataframe, kde se snaží najít podle vhodných podmínek vhodný vzor, podle kterého se
    budou ořezávat obrázky v přištích funkcí

    také to vrátí dataframe, který se dál použije k zapsání csv
    """
    # Inicializace proměnných pro maximální hodnotu důvěry, nejlepší název a další
    max_conf = float('-inf')
    best_name = None
    best_center_x = None
    best_center_y = None
    best_width = None
    best_height = None

    count_b = 0

    # Procházení řádků DataFrame
    for index, row in df.iterrows():
        # Získání hodnot z aktuálního řádku
        conf_value = float(row['conf'])
        bb_width = float(row['bb_width'])
        bb_height = float(row['bb_height'])
        smallBox = row['smallBox']

        # Kontrola podmínek pro šířku, výšku a smallBox
        if bb_width < 55 and bb_height < 55 and smallBox == "yes":
            # Pokud je aktuální hodnota důvěry vyšší než maximální hodnota, aktualizace maximální hodnoty a dalších proměnných
            if conf_value > max_conf:
                max_conf = conf_value
                best_name = row['name']
                best_center_x = row["center_x"]
                best_center_y = row["center_y"]
                best_width = row["real_width"]
                best_height = row["real_height"]
                count_b += 1

        # Aktualizace sloupce 'best' v aktuálním řádku
        df.at[index, 'best'] = count_b if conf_value == max_conf else 0

    # Vrácení potřebných hodnot
    return best_name, best_center_x, best_center_y, best_width, best_height, df



def df_write(best_name: str, df: pd.DataFrame, path_of_csv: str):
    """
    Touto funkcí zapisujeme, podle kterého bounding boxu jsme ořezali složku a informace v podobě CSV ukládáme
    do složky bounding_boxes.

    PŘEPSAT SLOUPEC
    """
    # Rozdělení názvu na části a vytvoření cesty pro výstupní CSV soubor
    mirror = best_name.split("_")
    output_name = os.path.join(path_of_csv, mirror[0], mirror[-1].replace(".txt", "") + ".csv")

    # Vytvoření složky, pokud neexistuje
    folder_path = os.path.join(path_of_csv, mirror[0])
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    # Najití největší hodnoty ve sloupci
    column_index = 13  # Index sloupce, ve kterém chcete provést operaci
    column_values = []

    for row in df.itertuples(index=False):
        try:
            value = float(row[column_index])
            column_values.append(value)
        except ValueError:
            # Pokud hodnota není číselná, ignorujeme ji
            pass

    max_value = max(column_values)

    # Nahrazení hodnot
    for i, row in enumerate(df.itertuples(index=False)):
        try:
            value = float(row[column_index])
            if value == max_value:
                df.iat[i, column_index] = '1'
            else:
                df.iat[i, column_index] = '0'
        except ValueError:
            # Pokud hodnota není číselná, ponecháme ji beze změny
            pass

    # Uložení změn do nového CSV souboru
    df.to_csv(output_name, index=False)
    print(output_name)


def prepare_cropping_img(best_name: str, orig_data_path: str)-> list:
    """
    tato funkce vytvoří seznam původních obrázků, které se ořežou a vrátí je jako list

    """

    # Inicializace proměnných pro název obrázku a základní adresář
    img_name = best_name
    base_dir = orig_data_path

    # Rozdělení názvu souboru a úprava části pro vytvoření cesty
    shards = img_name.split("_")
    shards[2] = shards[2].replace(".txt", "")
    img_path = base_dir + "Tray_" + shards[0] + "/" + shards[2] + "/"

    # Získání seznamu souborů v adresáři
    folder_path = img_path
    file_list = os.listdir(folder_path)

    # Filtrování souborů s příponou .png
    png_files = [file for file in file_list if file.endswith('.png')]

    # Vytvoření absolutních cest k souborům
    absolute_paths = [os.path.join(folder_path, file) for file in png_files]
    return absolute_paths

def cropping_img(best_center_x: str, best_center_y: str, best_width: str, best_height: str, absolute_paths):
    """
    tato funkce vyřeže obrázky z listu absolute:paths podle paratrů, které získáme ve funkci finding_BB_DF

    """
    for img in absolute_paths:
        # Načtení obrázku
        img_path = img
        image = cv2.imread(img_path)

        # Nastavení velikosti výřezu a souřadnic středu bounding boxu
        crop_size = (75, 75)
        bb_center_x = float(best_center_x)
        bb_center_y = float(best_center_y)
        img_h = float(best_height)
        img_w = float(best_width)

        # Výpočet souřadnic středu výřezu v pixelech
        center_x = int(bb_center_x * img_w)
        center_y = int(bb_center_y * img_h)

        # Určení souřadnic levého horního rohu výřezu
        top_left_x = max(center_x - crop_size[0] // 2, 0)
        top_left_y = max(center_y - crop_size[1] // 2, 0)

        # Vytvoření výřezu
        cropped_image = image[top_left_y:top_left_y + crop_size[1], top_left_x:top_left_x + crop_size[0]]

        # Cesta pro uložení výřezu
        save_adress_full = save_adress(img_path, base_save_adress)
        output_path = save_adress_full

        # Uložení výřezu
        cv2.imwrite(output_path, cropped_image)


def save_adress(img_path: str, base_save_adress: str) -> str:
    """
    vytvoření cesty, kam se uloží ořezané obrázky, používá se ve funkci cropping_img

    """
    # Rozdělení cesty k obrázku na části podle "/"
    parts = img_path.split("/")

    # Rozdělení poslední části (název souboru) podle "_"
    shards = parts[-1].split("_")

    # Nahrazení přípony ".png" prázdným řetězcem u poslední části
    shards[-1] = shards[-1].replace(".png", "")

    # Sestavení cesty pro uložení adresáře
    save_adress_path = base_save_adress + "/Tray_" + shards[-3] + "/" + shards[-1] + "/"

    # Pokud cesta neexistuje, vytvoření adresáře
    if not os.path.exists(save_adress_path):
        os.makedirs(save_adress_path)

    # Sestavení úplné cesty pro uložení souboru
    save_adress_full = save_adress_path + parts[-1]

    # Vrácení úplné cesty
    return save_adress_full


def print_folders_with_images(folder: str) -> list:
    """
    projde celý adresář včetně podadresářů a vrátí nám list všech adresářů ve složce
    a seřadí je podle abecedy

    """
    # Inicializace seznamu pro cesty adresářů obsahujících PNG obrázky
    directory_paths = []

    # Procházení složky a podadresářů
    for root, dirs, files in os.walk(folder):
        for directory in dirs:
            # Vytvoření cesty k aktuálnímu adresáři
            directory_path = os.path.join(root, directory)

            # Filtrování souborů s příponou .png v aktuálním adresáři
            png_files = [file for file in os.listdir(directory_path) if file.endswith('.png')]

            # Pokud adresář obsahuje PNG soubory, přidání jeho cesty do seznamu
            if png_files:
                directory_paths.append(directory_path)

    # Seřazení seznamu cest
    directory_paths.sort()

    # Vrácení seznamu cest adresářů obsahujících PNG obrázky
    return directory_paths


# EXECUTION

In [ ]:
folder = gdrive_path+"KIT ML 2/Data/1-TraysSegmentation/segmented"

#toto vypíše všechny podsložky ve složce
directory_paths = print_folders_with_images(folder)

#conf=0.6
model = YOLO(path_to_model)
delete(save_path)

for item in directory_paths:
    delete(save_path)
    path_to_imgs= item
    predictImgsSINGLEDirectory(path_to_model, path_to_imgs, conf,save_path,name)
    try:
        df = create_dataframe(label_path)
        df = add_size_to_dataframe(df, orig_data_path)
        df = bb_adding_smallbox(df)
        df = bb_smallBox_edit(df)
        best_name, best_center_x, best_center_y, best_width, best_height, df = finding_BB_DF(df)
        df_write(best_name, df, path_of_csv)
        absolute_paths = prepare_cropping_img(best_name, orig_data_path)
        cropping_img(best_center_x, best_center_y, best_width, best_height, absolute_paths)

    except Exception as e:
        print(f"An error occurred: {e}")
        continue


Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.3.0+cu121 CPU
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs
Results saved to /content/gdrive/Shareddrives/KIT ML 2/Data/cropped75-smazatt/runs/detect
36 labels saved to /content/gdrive/Shareddrives/KIT ML 2/Data/cropped75-smazatt/runs/detect/labels


/content/gdrive/Shareddrives/KIT ML 2/Data/cropped75-smazatt/runs/
detect
/content/gdrive/Shareddrives/KIT ML 2/Data/cropped75-smazatt//bounding_boxes/10/1-1.csv


Results saved to /content/gdrive/Shareddrives/KIT ML 2/Data/cropped75-smazatt/runs/detect
34 labels saved to /content/gdrive/Shareddrives/KIT ML 2/Data/cropped75-smazatt/runs/detect/labels


/content/gdrive/Shareddrives/KIT ML 2/Data/cropped75-smazatt/runs/
detect
/content/gdrive/Shareddrives/KIT ML 2/Data/cropped75-smazatt//bounding_boxes/10/1-2.csv


Results saved to /content/gdrive/Shareddrives/KIT ML 2/Data/cropped75-smazatt/runs/detect
34 labels saved to /content/gdrive/Shareddrives/KIT ML 2/Data/cropped75-smazatt/runs/detect/labels


...

Results saved to /content/gdrive/Shareddrives/KIT ML 2/Data/cropped75-smazatt/runs/detect
31 labels saved to /content/gdrive/Shareddrives/KIT ML 2/Data/cropped75-smazatt/runs/detect/labels


/content/gdrive/Shareddrives/KIT ML 2/Data/cropped75-smazatt/runs/
detect
/content/gdrive/Shareddrives/KIT ML 2/Data/cropped75-smazatt//bounding_boxes/10/1-7.csv


Results saved to /content/gdrive/Shareddrives/KIT ML 2/Data/cropped75-smazatt/runs/detect
30 labels saved to /content/gdrive/Shareddrives/KIT ML 2/Data/cropped75-smazatt/runs/detect/labels


/content/gdrive/Shareddrives/KIT ML 2/Data/cropped75-smazatt/runs/
detect
/content/gdrive/Shareddrives/KIT ML 2/Data/cropped75-smazatt//bounding_boxes/10/1-8.csv


KeyboardInterrupt: 